# Explorando la salida de la simulación

In [ ]:
import netCDF4 as nc
import numpy as np
import rasterio
import geopandas as gp
import pandas as pd
import xarray as xa
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
atm5 = xa.open_dataset("../../../../../DATOS/RegCM/__RUNS/RUN01/output/test_001_ATM.2010052500.nc")

In [ ]:
atm5.close()

In [ ]:
atm5.attrs

In [ ]:
atm5.variables

In [ ]:
surf05 = xa.open_dataset("../../../../../DATOS/RegCM/__RUNS/RUN01/output/test_001_SRF.2010052500.nc")
surf06 = xa.open_dataset("../../../../../DATOS/RegCM/__RUNS/RUN01/output/test_001_SRF.2010060100.nc")

In [ ]:
surf05.variables

In [ ]:
deptos = gp.read_file("../../../../../DATOS/GEO GT/gt-deptos-lowq-2017.geojson")

In [ ]:
lon = surf05.coords["xlon"]
lat = surf05.coords["xlat"]
lons,lats= np.meshgrid(lon-180,lat) # for this dataset, longitude is 0 through 360, so you need to subtract 180 to properly display on map

In [ ]:
lon.shape

In [ ]:
world = gp.read_file(
    gp.datasets.get_path('naturalearth_lowres')
)


In [ ]:
lon[::5, ::5]

In [ ]:
i = 34
plt.rcParams["figure.figsize"] = (10,10)
fig, ax = plt.subplots(figsize=(15, 15))
plcm=ax.pcolormesh(lon, lat, surf05["pr"].sel(time='2010-05-29T09:00:00.000000000'), shading="gouraud", vmax=0.010, vmin=0)
deptos[deptos.CODIGO!= "2300"].plot(color="#ffffff00", edgecolor="white", ax=ax)
xlim = ax.get_xlim()
ylim = ax.get_ylim()
world.plot(ax=ax, color="#ffffff00", edgecolor="yellow")
ax.set_xlim(xlim)
ax.set_ylim(ylim)
fig.colorbar(plcm)
qver = ax.quiver(lon[::3, ::3].values.flatten(), lat[::3, ::3].values.flatten(),
          surf05["uas"].sel(time='2010-05-29T09:00:00.000000000')[:, ::3, ::3].values.flatten(), 
          surf05["vas"].sel(time='2010-05-29T09:00:00.000000000')[:, ::3, ::3].values.flatten(), color="white",alpha=0.6)
qver.set_UVC(
          surf05["uas"].sel(time='2010-05-29T09:00:00.000000000')[:, ::3, ::3].values.flatten(), 
          surf05["vas"].sel(time='2010-05-29T09:00:00.000000000')[:, ::3, ::3].values.flatten())


In [ ]:
qver.set_UVC()

In [ ]:
precipData = xa.concat([surf05["pr"], surf06["pr"]], "time")

In [ ]:
precipData

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
fig = plt.figure()
ax = fig.gca()
pclm=ax.pcolormesh(lon, lat, precipData.sel(time=surf05.time[6])+1e-3, norm= mpl.colors.LogNorm(vmin=1e-3, vmax=precipData.max().values), shading="gouraud")
deptos[deptos.CODIGO!= "2300"].plot(color="#ffffff00", edgecolor="white", ax=ax)
xlim = ax.get_xlim()
ylim = ax.get_ylim()    
world.plot(ax=ax, color="#ffffff00", edgecolor="yellow")
ax.set_xlim(xlim)
ax.set_ylim(ylim)
fig.colorbar(pclm, shrink=0.5)

def init():
    return (pclm,)
    
# animation function. This is called sequentially
def animate(i):
    pclm.set_array(precipData.sel(time=precipData.time[i]).values.flatten() +1e-3)
    ax.set_title("Precipitation\n" + str(i) + " - "+ str(precipData.time.values[i]))
    return (pclm,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=88, interval=300)
                 
HTML(anim.to_html5_video())

In [ ]:
cloudData = xa.concat([surf05["clt"], surf06["clt"]], "time")
uasData = xa.concat([surf05["uas"], surf06["uas"]], "time")
vasData = xa.concat([surf05["vas"], surf06["vas"]], "time")

In [ ]:
# con total cloud fraction:
plt.rcParams["figure.figsize"] = (10,10)
fig, ax = plt.subplots()
pclm=ax.pcolormesh(lon, lat, surf05["clt"].sel(time=surf05.time[0]), vmax=100, vmin=0, shading="gouraud")
deptos[deptos.CODIGO!= "2300"].plot(color="#ffffff00", edgecolor="white", ax=ax)
xlim = ax.get_xlim()
ylim = ax.get_ylim()    
world.plot(ax=ax, color="#ffffff00", edgecolor="yellow")
ax.set_xlim(xlim)
ax.set_ylim(ylim)
fig.colorbar(pclm, shrink=0.5)
qver = ax.quiver(lon[::3, ::3].values.flatten(), lat[::3, ::3].values.flatten(),
          uasData.sel(time=cloudData.time[0])[:, ::3, ::3].values.flatten(), 
          vasData.sel(time=cloudData.time[0])[:, ::3, ::3].values.flatten(), color="#444444",alpha=0.6)

def init():
    return pclm,qver,
    
# animation function. This is called sequentially
def animate(i):
    pclm.set_array(cloudData.sel(time=cloudData.time[i]).values.flatten())
    ax.set_title("Total Cloud Fraction\n" + str(i) + " - "+ str(cloudData.time.values[i]))
    qver.set_UVC(
          uasData.sel(time=uasData.time[i])[:, ::3, ::3].values.flatten(), 
          vasData.sel(time=vasData.time[i])[:, ::3, ::3].values.flatten())

    return pclm, qver,

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(cloudData), interval=300)
                 
HTML(anim.to_html5_video())

In [ ]:
anim.save("Nubosidad-direccion del viento Agatha 20100525-20100605.mp4")